In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

logits = torch.randn(2, 3, 4) # batch_size = 2, seq_len = 3, vocab_size = 4
# print(logits)
# print("The shape of logits is: ", logits.shape)
# print("-"*100)

label = torch.randint(0, 4, (2, 3))
# print(label)
logits = logits.transpose(1, 2)
# print(logits)
# print("The shape of logits is: ", logits.shape)

# print("-"*100)
# print(F.cross_entropy(logits, label))
# print(F.cross_entropy(logits, label, reduction='none'))

tgt_len = torch.Tensor([2, 3]).to(torch.int32)
mask = torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len)-L)), 0) for L in tgt_len], dim=0)

# print(mask)
# print("The shape of mask is: ", mask.shape)

# for i in mask:
#     print(i.shape)

# 对padding部分的loss值mask掉，使得padding部分对loss的贡献为0
loss = F.cross_entropy(logits, label, reduction='none')
# print("Before maskd: ", loss)
# print("After masked: ", loss * mask)

# 事实上可以直接使用F.cross_entropy()的参数ignore_index
label_back = label
label_back[0, 2] = -100 # 手动对padding部分赋值为-100
print(label_back)
loss_new = F.cross_entropy(logits, label_back, reduction='none')
loss_ignore_index = F.cross_entropy(logits, label, ignore_index=-100, reduction='none') # F.cross_entropy()有参数ignore_index，其针对padding部分将其赋值为-100
print("使用F.cross_entropy()的ignore_index参数: ", '\n',loss_ignore_index, '\n')
print("不使用API的参数，手动为其赋值", '\n',loss_new)

tensor([[   3,    2, -100],
        [   1,    3,    1]])
使用F.cross_entropy()的ignore_index参数:  
 tensor([[0.1995, 2.2872, 0.0000],
        [0.6224, 1.9133, 1.7468]]) 

不使用API的参数，手动为其赋值 
 tensor([[0.1995, 2.2872, 0.0000],
        [0.6224, 1.9133, 1.7468]])
